In [1]:
%config ZMQInteractiveShell.ast_node_interactivity='all'
%matplotlib inline
import warnings;warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.preprocessing import Imputer, LabelEncoder
from sklearn import metrics
import os
from scipy import stats as spstats

In [2]:
### flag
fill_data = True

In [3]:
#### load data files for d1 d2 ###

# Generate data file name list
file_names = ["Application.csv","Personas.csv","History_Application.csv","History_Payment.csv"]

# load data files
d01 = pd.read_csv(f'./data/train_{file_names[0]}')
d02 = pd.read_csv(f'./data/train_{file_names[1]}')

# Change file name prefix here and you are good to go:D
prefix = "B_"
# load data files
d05 = pd.read_csv(f'./data/{prefix}{file_names[0]}')
d06 = pd.read_csv(f'./data/{prefix}{file_names[1]}')

# Concat
d1 = pd.concat([d01, d05], axis=0)
d2 = pd.concat([d02, d06], axis=0)


In [4]:
#### load data files for d3 ###
d03 = pd.read_csv(f'./data/train_{file_names[2]}')
d07 = pd.read_csv(f'./data/{prefix}{file_names[2]}')
d3 = pd.concat([d03, d07], axis=0)

In [5]:
#### load data files for d4 ###
d04 = pd.read_csv(f'./data/train_{file_names[3]}')
d08 = pd.read_csv(f'./data/{prefix}{file_names[3]}')
d4 = pd.concat([d04, d08], axis=0)

In [6]:
### Functions

# 取一个列的众数
def get_mode(df, col):
    return pd.Series(df[col]).mode()[0]

# 置多个列的空值为所属列0
def fill_zero(df, cols):
    dict = {}
    for col in cols:
        dict[col] = 0
    return df.fillna(dict)

# 置多个列的空值为所属列的平均数
def fill_mean(df, cols):
    dict = {}
    for col in cols:
        dict[col] = df[col].mean()
    return df.fillna(dict)

# 置多个列的空值为所属列的median
def fill_median(df, cols):
    dict = {}
    for col in cols:
        dict[col] = df[col].median()
    return df.fillna(dict)

# 置多个列的空值为所属列的众数
def fill_mode(df, cols):
    dict = {}
    for col in cols:
        dict[col] = get_mode(df, col)
    return df.fillna(dict)

# 把多个离散型的列转化为one-hot编码列
def dummy_catgorical(df, cols, keep_n_cats):
    print('initial num of cols: ' + str(len(df.columns.to_list())))
    for col in cols:
        # 跳过离散变量太多的
        idx_list = df[col].value_counts().index.to_list()
        print(col+'has'+str(len(idx_list)))
        if (len(idx_list) > keep_n_cats):
            print(' col含有过多值, skipping')
        else:
            # 生成one-hot编码列
            df = pd.get_dummies(df, columns=[col])
    return df

# 打印一个df的na值
def print_na(df):
    # 检查d12 nan值
    df_na = df.isna().sum()
    for idx in df_na.index.to_list():
        if (df_na[idx]!=0):
            print(str(idx)+' '+str(df_na[idx]))

def print_val(feat):
    return feat.value_counts(dropna=False)

In [7]:
### intersection
# ''' d01&d02 '''
# id_train = set(d01['申请编号'])
# len(id_train)
# id_test = set(d02['申请编号'])
# len(id_test)
# inter = id_train.intersection(id_test)
# len(inter)

In [8]:
### Merge d1 d2 ###
uid = '申请编号'
d12 = pd.merge(d1, d2, how='inner', on=uid)
d12.shape

(161469, 62)

In [9]:
print_val(d12['单位类型'])

# d12.columns.to_list()

5     36669
57    26813
42    20609
33     8850
30     5828
4      5632
11     5442
39     4614
51     4218
7      3736
28     3669
3      3192
55     2858
47     1887
26     1791
20     1756
40     1726
13     1578
16     1455
31     1370
2      1342
1      1341
34     1185
53     1181
35     1138
41     1022
38     1003
46      999
43      836
56      696
24      694
54      682
14      566
12      541
9       499
21      477
50      317
22      312
44      310
10      304
27      300
19      240
0       208
36      207
17      189
8       184
29      171
45      169
32      159
6       138
52      103
15       58
23       56
37       38
18       36
48       35
49       27
25       13
Name: 单位类型, dtype: int64

In [10]:
### Outlier：Get ###
low = .01
high = .99
quant_df = d12.drop([uid], axis=1).quantile([low, high]).T
quant_df.to_csv('./tmp/1_d12_outlier.csv')

### Outlier：Apply ###

outliers = pd.read_csv('./0_outlier1.csv', index_col=0)
outliers.shape
outliers.head()

def set_outlier(col):
    if col < col_min:
        col = col_min
    elif col > col_max:
        col = col_max
    return col

# def set_outlier(col):
#     if col < col_min or col > col_max:
#         col = np.NaN
#     return col
    
for col in outliers.index:
# for col in '出生日期距申请日期天数', '社交圈违约信息1_2':
    col_min = outliers.loc[col, 'min']
    col_max = outliers.loc[col, 'max']
    d12[col] = d12[col].apply(set_outlier)
    
# d12.to_csv(f'./tmp/2_eda_outlier.csv')

(17, 2)

,min,max
信用额度,100000,1000000
出生日期距申请日期天数,-50000,-6570
商品价格,100000,1000000
外部评分,0,1
孩子个数,0,5


In [11]:
print_na(d1)
''''''
print_na(d2)

贷款年金 7
商品价格 137
陪同申请人 670
外部评分 90777
贷款申请前1小时内征信查询次数 22203
贷款申请前1天内征信查询次数 22203
贷款申请前1周内征信查询次数 22203
贷款申请前1个月内征信查询次数 22203
贷款申请前1个季度内征信查询次数 22203
贷款申请前1年内征信查询次数 22203


''

有车时间 106452
职业 48509
家庭成员数 1
居住信息1 107790
居住信息2 113104
居住信息3 81552
居住信息4 78505
社交圈违约信息2_2 515
社交圈违约信息2_1 515
社交圈违约信息1_2 515
社交圈违约信息1_1 515


In [12]:
# d12.loc[d12['是否有车']==0,'有车时间']=0

# print_val(d12['孩子个数'])

# print_val(d12['有车月数'])

# d1['陪同申请人'].median()

# d12['有车月数'] = round(d12['有车时间']/30)


In [12]:
### d12 异常值处理 & 自定义特征

# d1 #

# 商品价格
d12['商品价格'] = round(d12['商品价格']/10000)
d12['商品价格小于42'] = d12['商品价格'].apply(lambda x: 1 if ((not np.isnan(x))&(x<42)) else 0)
# 外部评分
d12['有外部评分'] = d12['外部评分'].apply(lambda x: 0 if np.isnan(x) else 1)

# d2 #

# 有车时间
d12.loc[d12['是否有车']==0,'有车时间']=0
d12['有车月数'] = round(d12['有车时间']/30)

# dummy with nan flag
d12 = pd.get_dummies(d12,dummy_na=True,columns=['职业','陪同申请人'])

cell_days = '最近一次换手机号码距申请日天数'
cell_mons = '最近一次换手机号月数'
d12.loc[(d12[cell_days]>0), cell_days] = np.NaN
d12[cell_mons] = d12[cell_days].apply(lambda x: np.NaN if np.isnan(x) else round(x/30)) 
d12['一月内换过手机号'] = d12[cell_days].apply(lambda x: 0 if ((np.isnan(x))|(x>-30)) else 1)

bday_yrs = '出生日期距申请日期天数'
d12[bday_yrs] = round(d12[bday_yrs]/365)
d12['25岁以下'] = d12[bday_yrs].apply(lambda x: 1 if (x > -25) else 0)
d12['25岁到30岁'] = d12[bday_yrs].apply(lambda x: 1 if ((x > -30)|(x <= -25)) else 0)
d12['30岁到40岁'] = d12[bday_yrs].apply(lambda x: 1 if ((x > -40)|(x <= -30)) else 0)
d12['40岁到50岁'] = d12[bday_yrs].apply(lambda x: 1 if ((x > -50)|(x <= -40)) else 0)
d12['50岁以上'] = d12[bday_yrs].apply(lambda x: 1 if (x <= -50) else 0)

work_yrs = '工作日期距申请日期天数'
d12.loc[(d12[work_yrs]==292204), work_yrs] = np.NaN
d12.loc[(d12[work_yrs]>0), work_yrs] = np.NaN
d12[work_yrs] = d12[work_yrs].apply(lambda x: np.NaN if np.isnan(x) else round(x/365)) 
d12['工龄小于2年'] = d12[work_yrs].apply(lambda x: 1 if ((not np.isnan(x))&(x>-2)) else 0)

id_mons = '身份认证日期距申请日期天数'
d12.loc[(d12[id_mons]>0), id_mons] = np.NaN
d12[id_mons] = d12[id_mons].apply(lambda x: np.NaN if np.isnan(x) else round(x/30)) 
d12['注册日期距申请日期月数'] = round(d12['注册日期距申请日期天数']/30)

d12['居住地人口密集度'] = round(d12['居住地人口密集度'],3)

# 客户居住地评分
addr_score_cols = ['客户居住地评分1','客户居住地评分2']
d12['客户居住地评分sum'] = d12[addr_score_cols].sum(axis=1,min_count=0)
d12['客户居住地评分mean'] = d12['客户居住地评分sum'] / len(addr_score_cols)

# 地址是否一致标志
addr_algn_cols = [
    '地址是否一致标志1',
    '地址是否一致标志2',
    '地址是否一致标志3',
    '地址是否一致标志4',
    '地址是否一致标志5',
    '地址是否一致标志6'
]
d12['地址是否一致标志sum'] = d12[addr_algn_cols].sum(axis=1,min_count=0)
d12['地址是否一致标志mean'] = d12['地址是否一致标志sum'] / len(addr_algn_cols)

# 居住信息
addr_line_cols = [
    '居住信息1',
    '居住信息2',
    '居住信息3',
    '居住信息4'
]
for col in addr_line_cols:   
    d12['有' + col] = d12[col].apply(lambda x: 0 if np.isnan(x) else 1 )
    d12[col] = round(d12[col],2)
addr_line_e_cols = [
    '有居住信息1',
    '有居住信息2',
    '有居住信息3',
    '有居住信息4'
]
d12['居住信息条数'] = d12[addr_line_e_cols].sum(axis=1,min_count=0)

d12['居住信息sum'] = d12[addr_line_cols].sum(axis=1,min_count=0)
d12['居住信息mean'] = d12['居住信息sum'] / len(addr_line_cols)

d12 = fill_mode(d12, ['社交圈违约信息2_2', '社交圈违约信息2_1', '社交圈违约信息1_2', '社交圈违约信息1_1'])
d12['社交圈违约信息2sum'] = (d12['社交圈违约信息2_2'] + d12['社交圈违约信息2_1'])
d12['社交圈违约信息2mean'] = d12['社交圈违约信息2sum'] / 2
d12['社交圈违约信息1sum'] = (d12['社交圈违约信息1_2'] + d12['社交圈违约信息1_1'])
d12['社交圈违约信息1mean'] = d12['社交圈违约信息1sum'] / 4

nan_to_median = [
    '贷款年金',
    '商品价格',
    '外部评分',
    '有车时间',
    '有车月数',
    '家庭成员数',
    cell_days,
    cell_mons,
    work_yrs,
    id_mons
]

cred_inq_cols=[
'贷款申请前1小时内征信查询次数',
'贷款申请前1天内征信查询次数',
'贷款申请前1周内征信查询次数',
'贷款申请前1个月内征信查询次数',
'贷款申请前1个季度内征信查询次数',
'贷款申请前1年内征信查询次数'
]

nan_to_median = nan_to_median + cred_inq_cols + addr_line_cols

if (fill_data):
    d12 = fill_median(d12, nan_to_median)
    d12 = fill_zero(d12, ['商品价格小于42'])
    
d12.head()

# d12['最近一次换手机号码距申请日天数'].value_counts(dropna=False)
# d12['工作日期距申请日期天数'].value_counts(dropna=False)
# d12['身份认证日期距申请日期天数'].value_counts(dropna=False)
# d12['有车时间'].value_counts(dropna=False)

,申请编号,贷款类型,信用额度,贷款年金,商品价格,出生日期距申请日期天数,工作日期距申请日期天数,注册日期距申请日期天数,身份认证日期距申请日期天数,是否提供手机号,...,有居住信息2,有居住信息3,有居住信息4,居住信息条数,居住信息sum,居住信息mean,社交圈违约信息2sum,社交圈违约信息2mean,社交圈违约信息1sum,社交圈违约信息1mean
0,0,0,460190.889355,17463.042019,42.0,-28.0,-1.0,-5421,-110,1,...,1,1,1,4,0.91,0.2275,0.0,0.0,0.0,0.00
1,1,0,424370.659603,15585.046388,38.0,-38.0,-3.0,-3387,-28,1,...,0,0,0,0,0.00,0.0000,0.0,0.0,0.0,0.00
2,2,0,469330.587153,18837.558252,42.0,-37.0,-6.0,-3870,-6,1,...,1,1,1,4,0.85,0.2125,0.0,0.0,0.0,0.00
3,3,0,464188.353140,16754.587069,41.0,-45.0,-4.0,-970,-97,0,...,0,0,0,0,0.00,0.0000,1.0,0.5,1.0,0.25
4,5,0,434196.988738,16165.009200,39.0,-49.0,-30.0,-9350,-120,1,...,0,1,1,2,0.23,0.0575,0.0,0.0,0.0,0.00


In [13]:
d12.shape
'''d12 nan值'''
print_na(d12)
d12.columns.to_list()

(161469, 114)

'd12 nan值'

['申请编号',
 '贷款类型',
 '信用额度',
 '贷款年金',
 '商品价格',
 '出生日期距申请日期天数',
 '工作日期距申请日期天数',
 '注册日期距申请日期天数',
 '身份认证日期距申请日期天数',
 '是否提供手机号',
 '是否提供电话',
 '手机号是否有效',
 '是否提供email',
 '申请周内日',
 '申请时点',
 '外部评分',
 '申请人是否额外提供了文件2',
 '申请人是否额外提供了文件3',
 '申请人是否额外提供了文件4',
 '申请人是否额外提供了文件5',
 '申请人是否额外提供了文件6',
 '申请人是否额外提供了文件7',
 '申请人是否额外提供了文件8',
 '申请人是否额外提供了文件9',
 '贷款申请前1小时内征信查询次数',
 '贷款申请前1天内征信查询次数',
 '贷款申请前1周内征信查询次数',
 '贷款申请前1个月内征信查询次数',
 '贷款申请前1个季度内征信查询次数',
 '贷款申请前1年内征信查询次数',
 '性别',
 '是否有车',
 '是否有房',
 '孩子个数',
 '客户收入',
 '收入类型',
 '教育程度',
 '婚姻状态',
 '居住状态',
 '居住地人口密集度',
 '有车时间',
 '家庭成员数',
 '居住信息1',
 '居住信息2',
 '居住信息3',
 '居住信息4',
 '客户居住地评分1',
 '客户居住地评分2',
 '地址是否一致标志1',
 '地址是否一致标志2',
 '地址是否一致标志3',
 '地址是否一致标志4',
 '地址是否一致标志5',
 '地址是否一致标志6',
 '单位类型',
 '社交圈违约信息2_2',
 '社交圈违约信息2_1',
 '社交圈违约信息1_2',
 '社交圈违约信息1_1',
 '最近一次换手机号码距申请日天数',
 '商品价格小于42',
 '有外部评分',
 '有车月数',
 '职业_0.0',
 '职业_1.0',
 '职业_2.0',
 '职业_3.0',
 '职业_4.0',
 '职业_5.0',
 '职业_6.0',
 '职业_7.0',
 '职业_8.0',
 '职业_9.0',
 '职业_10.0',
 '职业_11.0',
 '职业_13.0',
 '职业_14.0',
 '职业_15.0'

In [15]:
print_na(d12)

In [16]:
### d12 类型变量转dummy列 ###

cat_to_dummy = [
    '收入类型',
    '教育程度',
    '居住状态',
    '单位类型'
]

d12 = dummy_catgorical(d12, cat_to_dummy, 60)
d12.shape



initial num of cols: 114
收入类型has8
教育程度has5
居住状态has6
单位类型has58


(161469, 187)

In [17]:
d3.columns.to_list()
    
# d12.to_csv('./tmp/d12.csv')

['历史申请编号',
 '申请编号',
 '贷款类型',
 '贷款年金',
 '申请额度',
 '信用额度',
 '首付金额',
 '商品价格',
 '申请周内日',
 '申请时点',
 '是否为最后一次申请记录',
 '是否为当天最后一次申请记录',
 '首付率',
 '利率1',
 '利率2',
 '贷款用途',
 '合同状态',
 '历史贷款授信距本次申请时间',
 '付款方式',
 '拒绝原因',
 '陪同申请人',
 '客户类型',
 '商品类别',
 '组合类型',
 '产品类型',
 '获客渠道',
 '销售区域',
 '行业',
 '贷款期数',
 '收益级别',
 '产品组合',
 '首付时间']

In [14]:
### d3过滤

# 删除特征'是否为最后一次申请记录'
if ('是否为最后一次申请记录' in d3.columns):
    d3.drop(columns='是否为最后一次申请记录',inplace=True)

In [15]:
### d3数据清洗 - 金额型 ###

d3copy = d3.copy()

# 异常值置NaN
d3.loc[(d3['首付时间'] == 292204.0), '首付时间'] = np.NaN
d3.loc[(d3['首付金额'] == 11), '首付金额'] = np.NaN
d3.loc[(d3['贷款年金'] == 11), '贷款年金'] = np.NaN
d3.loc[(d3['商品价格'] == 11), '商品价格'] = np.NaN
d3.loc[(d3['申请额度'] == 11), '申请额度'] = np.NaN
d3.loc[(d3['信用额度'] == 11), '信用额度'] = np.NaN

d3['信用额度差'] = d3['信用额度'] - d3['申请额度']
d3['信用额度比'] = round(d3['信用额度']/d3['申请额度'],2)

d3['有贷款年金'] = d3['贷款年金'].apply(lambda x: 0 if np.isnan(x) else 1 )
d3['有商品价格'] = d3['商品价格'].apply(lambda x: 0 if np.isnan(x) else 1 )
d3['有信用额度'] = d3['信用额度'].apply(lambda x: 0 if np.isnan(x) else 1 )
d3['贷款年金'] = d3['贷款年金'].apply(lambda x: np.NaN if np.isnan(x) else round(x/100))
d3['申请额度'] = d3['申请额度'].apply(lambda x: np.NaN if np.isnan(x) else round(x/1000)) 
d3['信用额度'] = d3['信用额度'].apply(lambda x: np.NaN if np.isnan(x) else round(x/1000)) 

hist_app_days = '历史贷款授信距本次申请时间'
hist_app_mons = '历史贷款授信月数'
d3['有' + hist_app_days] = d3[hist_app_days].apply(lambda x: 0 if x>0 else 1 )
d3.loc[(d3[hist_app_days]>0), hist_app_days] = 0
d3[hist_app_mons] = round(d3[hist_app_days]/30)

dpay_days = '首付时间'
d3['有' + dpay_days] = d3[dpay_days].apply(lambda x: 0 if x>0 else 1 )
d3.loc[(d3[dpay_days]>0), dpay_days]=0
d3['首付月数'] = round(d3[dpay_days]/30)


cat_to_dummy = [
    '拒绝原因',
    '客户类型', #!
    '商品类别',
    '组合类型',
    '产品类型',
    '获客渠道',
    '行业',
    '收益级别', #!
    '产品组合',
]

d3 = dummy_catgorical(d3, cat_to_dummy, 27)

d3_agg_dict = {
    '历史申请编号':['count'],
    '贷款年金':['mean', 'max', 'min'],
    '申请额度':['mean', 'max', 'min'],
    '信用额度':['mean', 'max', 'min'],
#     '首付金额':['mean', 'max', 'min'],
    '商品价格':['mean', 'max', 'min'],
    '信用额度差':['mean'],
    '信用额度比':['mean', 'max'],
    # 数值型 - 时长 
    '历史贷款授信距本次申请时间':['mean', 'max', 'min'],
    '历史贷款授信月数':['mean', 'max', 'min'],
#     '首付时间':['mean', 'max', 'min'],
#     '首付月数':['mean', 'max', 'min'],
    # 概率型
#     '首付率':['mean', 'max', 'min']
}
    
# 离散型
cat_agg = ['sum', 'mean']
for col1 in d3.columns:
    for col2 in cat_to_dummy:
        if (col2 in col1):
            d3_agg_dict[col1] = cat_agg

# 非拒合同状态group
d3 = d3.groupby('申请编号').agg(d3_agg_dict)
# 还原列名
d3.columns = pd.Series(d3.columns.tolist()).apply(pd.Series).sum(axis=1)
name_list = {}
for col in d3.columns:
    name_list[col] = 'd3'+col
d3.rename(columns=name_list,inplace=True)

d3.shape
    
'''d3 nan值'''
print_na(d3)

initial num of cols: 40
拒绝原因has9
客户类型has4
商品类别has27
组合类型has5
产品类型has3
获客渠道has8
行业has11
收益级别has5
产品组合has17


,d3历史申请编号count,d3贷款年金mean,d3贷款年金max,d3贷款年金min,d3申请额度mean,d3申请额度max,d3申请额度min,d3信用额度mean,d3信用额度max,d3信用额度min,...,d3产品组合_13.0sum,d3产品组合_13.0mean,d3产品组合_14.0sum,d3产品组合_14.0mean,d3产品组合_15.0sum,d3产品组合_15.0mean,d3产品组合_16.0sum,d3产品组合_16.0mean,d3产品组合_17.0sum,d3产品组合_17.0mean
申请编号,,,,,,,,,,,,,,,,,,,,,
0,1,11.000000,11.0,11.0,10.00,10.0,10.0,9.000000,9.0,9.0,...,0,0.0,1,1.0,0,0.0,0,0.0,0,0.0
1,2,8.000000,10.0,6.0,5.00,6.0,4.0,6.500000,8.0,5.0,...,0,0.0,2,1.0,0,0.0,0,0.0,0,0.0
2,10,42.142857,74.0,14.0,49.00,79.0,28.0,50.750000,79.0,28.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
3,1,49.000000,49.0,49.0,51.00,51.0,51.0,49.000000,49.0,49.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
4,9,17.666667,26.0,4.0,25.00,40.0,3.0,25.555556,40.0,3.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182975,3,12.333333,14.0,11.0,9.00,12.0,6.0,15.333333,28.0,6.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
182976,2,27.000000,27.0,27.0,24.00,24.0,24.0,25.000000,25.0,25.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
182977,1,5.000000,5.0,5.0,5.00,5.0,5.0,5.000000,5.0,5.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0


(152954, 200)

'd3 nan值'

贷款年金mean 206
贷款年金max 206
贷款年金min 206
申请额度mean 533
申请额度max 533
申请额度min 533
信用额度mean 107
信用额度max 107
信用额度min 107
商品价格mean 533
商品价格max 533
商品价格min 533
信用额度差mean 533
信用额度比mean 533
信用额度比max 533


In [16]:
if (fill_data):
    for col in d3.columns:
        d3[col].fillna(d3[col].median(),inplace=True)

In [17]:
# d3 贷款类型(0,1,A)不为A、申请额度不等于11的、拒绝原因为7的
# group by 申请编号
# 补差、one-hot编码、计算自定义特征


#过滤
d3_type01 = d3copy.loc[(d3copy['贷款类型']!='A')&(d3copy['申请额度']!=11)&(d3copy['拒绝原因']==7),:]
d3_type01.shape

#one-hot编码
cat_to_dummy = [
    '客户类型', #!
    '收益级别' #!
]
d3_type01 = dummy_catgorical(d3_type01, cat_to_dummy, 30)

#计算简单特征
d3_type01['信用额度差'] = d3_type01['信用额度'] - d3_type01['申请额度']
d3_type01['信用额度比'] = round(d3_type01['信用额度']/d3_type01['申请额度'],2)
d3_type01['年金乘期数'] = d3_type01['贷款年金'] * d3_type01['贷款期数']
d3_type01['期数*24'] = d3_type01['贷款期数'] * 24 # 假设贷款总天数
d3_type01['贷款剩余天数'] = d3_type01['期数*24'] + d3_type01['历史贷款授信距本次申请时间'] # 假设贷款剩余天数
d3_type01['贷款未还完'] = d3_type01['贷款剩余天数'].apply(lambda x: 1 if x>0 else 0 )

#计算复杂特征
d3_type01_agg = {
    '历史申请编号':['count'],
    '申请额度':['mean', 'max'],
    '信用额度':['mean', 'max'],
    '商品价格':['mean'],
    '信用额度差':['mean'],
    '信用额度比':['mean', 'max'],
    '贷款剩余天数':['max'], #最长未还天数
    '贷款未还完':['sum']
}
# 离散型
cat_agg = ['sum', 'mean']
for col1 in d3_type01.columns:
    for col2 in cat_to_dummy:
        if (col2 in col1):
            d3_type01_agg[col1] = cat_agg

d3_type01 = d3_type01.groupby('申请编号').agg(d3_type01_agg)
# 还原列名
d3_type01.columns = pd.Series(d3_type01.columns.tolist()).apply(pd.Series).sum(axis=1)
name_list = {}
for col in d3_type01.columns:
    name_list[col] = 'd3_tp01'+col
d3_type01.rename(columns=name_list,inplace=True)

if (fill_data):
    d3_type01 = fill_median(d3_type01,['贷款剩余天数max'])
    
d3_type01.shape
d3_type01.head()

(165364, 31)

initial num of cols: 31
客户类型has4
收益级别has5


,d3_tp01历史申请编号count,d3_tp01申请额度mean,d3_tp01申请额度max,d3_tp01信用额度mean,d3_tp01信用额度max,d3_tp01商品价格mean,d3_tp01信用额度差mean,d3_tp01信用额度比mean,d3_tp01信用额度比max,d3_tp01贷款剩余天数max,...,d3_tp01收益级别_0sum,d3_tp01收益级别_0mean,d3_tp01收益级别_1sum,d3_tp01收益级别_1mean,d3_tp01收益级别_2sum,d3_tp01收益级别_2mean,d3_tp01收益级别_3sum,d3_tp01收益级别_3mean,d3_tp01收益级别_4sum,d3_tp01收益级别_4mean
申请编号,,,,,,,,,,,,,,,,,,,,,
2,3,46211.0,59411,47082.00,59411.0,46211.0,871.00,1.023333,1.07,500.0,...,0,0.0,3,1.0,0,0.0,0,0.0,0,0.0
4,2,16247.0,20603,18891.50,23321.0,16247.0,2644.50,1.175000,1.22,-1095.0,...,0,0.0,2,1.0,0,0.0,0,0.0,0,0.0
5,2,35651.0,47531,36043.00,47531.0,35651.0,392.00,1.015000,1.03,-170.0,...,1,0.5,0,0.0,0,0.0,0,0.0,1,0.5
8,2,110891.0,142571,123101.50,152356.0,110891.0,12210.50,1.125000,1.18,982.0,...,0,0.0,2,1.0,0,0.0,0,0.0,0,0.0
11,2,60203.0,80003,66002.00,85810.0,60203.0,5799.00,1.105000,1.14,117.0,...,0,0.0,0,0.0,0,0.0,2,1.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182964,2,37631.0,39611,44526.50,53402.0,37631.0,6895.50,1.175000,1.35,-230.0,...,0,0.0,0,0.0,0,0.0,0,0.0,2,1.0
182966,5,24087.4,39611,28953.60,43531.0,24087.4,4866.20,1.204000,1.31,178.0,...,0,0.0,1,0.2,0,0.0,1,0.2,3,0.6
182973,2,39611.0,55451,45065.00,59951.0,39611.0,5454.00,1.175000,1.27,-42.0,...,0,0.0,1,0.5,0,0.0,0,0.0,1,0.5


(81362, 29)

,历史申请编号count,申请额度mean,申请额度max,信用额度mean,信用额度max,商品价格mean,信用额度差mean,信用额度比mean,信用额度比max,贷款剩余天数max,...,收益级别_0sum,收益级别_0mean,收益级别_1sum,收益级别_1mean,收益级别_2sum,收益级别_2mean,收益级别_3sum,收益级别_3mean,收益级别_4sum,收益级别_4mean
申请编号,,,,,,,,,,,,,,,,,,,,,
2,3,46211.0,59411,47082.0,59411.0,46211.0,871.0,1.023333,1.07,500.0,...,0,0.0,3,1.0,0,0.0,0,0.0,0,0.0
4,2,16247.0,20603,18891.5,23321.0,16247.0,2644.5,1.175000,1.22,-1095.0,...,0,0.0,2,1.0,0,0.0,0,0.0,0,0.0
5,2,35651.0,47531,36043.0,47531.0,35651.0,392.0,1.015000,1.03,-170.0,...,1,0.5,0,0.0,0,0.0,0,0.0,1,0.5
8,2,110891.0,142571,123101.5,152356.0,110891.0,12210.5,1.125000,1.18,982.0,...,0,0.0,2,1.0,0,0.0,0,0.0,0,0.0
11,2,60203.0,80003,66002.0,85810.0,60203.0,5799.0,1.105000,1.14,117.0,...,0,0.0,0,0.0,0,0.0,2,1.0,0,0.0


In [19]:
d3 = pd.merge(d3, d3_type01, how='left', on='申请编号')

if (fill_data):
    d3.fillna(0,inplace=True)

d3.shape
print_na(d3)

(152954, 229)

In [22]:
# d3 合同状态(0,1,2,3;2=拒绝)不为2、申请额度不等于11的
# group by 申请编号
# 补差、one-hot编码、计算自定义特征

#过滤
d3_state_ne2 = d3copy.loc[(d3copy['合同状态']!='2')&(d3copy['申请额度']!=11),:]
d3_state_ne2.shape

#one-hot编码
cat_to_dummy = [
    '客户类型', #!
    '收益级别' #!
]
d3_state_ne2 = dummy_catgorical(d3_state_ne2, cat_to_dummy, 30)

#计算特征
d3_state_ne2['信用额度差'] = d3_state_ne2['信用额度'] - d3_state_ne2['申请额度']
d3_state_ne2['信用额度比'] = round(d3_state_ne2['信用额度']/d3_state_ne2['申请额度'],2)
d3_state_ne2['年金乘期数'] = d3_state_ne2['贷款年金'] * d3_state_ne2['贷款期数']
d3_state_ne2['期数*24'] = d3_state_ne2['贷款期数'] * 24 # 假设贷款总天数
d3_state_ne2['贷款剩余天数'] = d3_state_ne2['期数*24'] + d3_state_ne2['历史贷款授信距本次申请时间'] # 假设贷款剩余天数
d3_state_ne2['贷款未还完'] = d3_state_ne2['贷款剩余天数'].apply(lambda x: 1 if x>0 else 0 )

#计算复杂特征

d3_state_ne2_agg = {
    '历史申请编号':['count'],
    '申请额度':['mean', 'max'],
    '信用额度':['mean', 'max'],
    '商品价格':['mean'],
    '信用额度差':['mean'],
    '信用额度比':['mean', 'max'],
    '贷款剩余天数':['max'], #最长未还天数
    '贷款未还完':['sum']
}
# 离散型
cat_agg = ['sum', 'mean']
for col1 in d3_state_ne2.columns:
    for col2 in cat_to_dummy:
        if (col2 in col1):
            d3_state_ne2_agg[col1] = cat_agg

d3_state_ne2 = d3_state_ne2.groupby('申请编号').agg(d3_state_ne2_agg)
# 还原列名
d3_state_ne2.columns = pd.Series(d3_state_ne2.columns.tolist()).apply(pd.Series).sum(axis=1)
name_list = {}
for col in d3_state_ne2.columns:
    name_list[col] = 'd3_st!2'+col
d3_state_ne2.rename(columns=name_list,inplace=True)

if (fill_data):
    d3_state_ne2 = fill_median(d3_state_ne2,['贷款剩余天数max'])
    
d3_state_ne2.shape
d3_state_ne2.head()

(572358, 31)

initial num of cols: 31
客户类型has4
收益级别has5


(152421, 29)

,历史申请编号count,申请额度mean,申请额度max,信用额度mean,信用额度max,商品价格mean,信用额度差mean,信用额度比mean,信用额度比max,贷款剩余天数max,...,收益级别_0sum,收益级别_0mean,收益级别_1sum,收益级别_1mean,收益级别_2sum,收益级别_2mean,收益级别_3sum,收益级别_3mean,收益级别_4sum,收益级别_4mean
申请编号,,,,,,,,,,,,,,,,,,,,,
0,1,9507.000000,9507,9403.000000,9403.0,9507.000000,-104.000000,0.990000,0.99,-1438.0,...,0,0.000,1,1.000000,0,0.0,0,0.000000,0,0.000000
1,2,4992.000000,6101,6360.000000,7766.0,4992.000000,1368.000000,1.275000,1.28,28.0,...,0,0.000,2,1.000000,0,0.0,0,0.000000,0,0.000000
2,8,49016.000000,79211,50714.625000,79211.0,49016.000000,1698.625000,1.032500,1.14,687.0,...,3,0.375,5,0.625000,0,0.0,0,0.000000,0,0.000000
3,1,51095.000000,51095,49020.000000,49020.0,51095.000000,-2075.000000,0.960000,0.96,-73.0,...,0,0.000,0,0.000000,0,0.0,0,0.000000,1,1.000000
4,9,24851.222222,39611,25470.777778,39611.0,24851.222222,619.555556,1.035556,1.22,-735.0,...,0,0.000,4,0.444444,0,0.0,1,0.111111,4,0.444444


In [87]:
d3 = pd.merge(d3, d3_state_ne2, how='left', on='申请编号')

if (fill_data):
    d3.fillna(0,inplace=True)

d3.shape
print_na(d3)

(152954, 258)

In [88]:
# d3 合同状态(0,1,2,3;2=拒绝)为2、申请额度不等于11的
# group by 申请编号
# 补差、one-hot编码、计算自定义特征

#过滤
d3_state_2 = d3copy.loc[(d3copy['合同状态']==2)&(d3copy['申请额度']!=11),:]
d3_state_2.shape

#one-hot编码
cat_to_dummy = [
    '客户类型', #!
    '收益级别' #!
]
d3_state_2 = dummy_catgorical(d3_state_2, cat_to_dummy, 30)

#计算特征
d3_state_2['信用额度差'] = d3_state_2['信用额度'] - d3_state_2['申请额度']
d3_state_2['信用额度比'] = round(d3_state_2['信用额度']/d3_state_2['申请额度'],2)
d3_state_2['年金乘期数'] = d3_state_2['贷款年金'] * d3_state_2['贷款期数']
d3_state_2['期数*24'] = d3_state_2['贷款期数'] * 24 # 假设贷款总天数
d3_state_2['贷款剩余天数'] = d3_state_2['期数*24'] + d3_state_2['历史贷款授信距本次申请时间'] # 假设贷款剩余天数
d3_state_2['贷款未还完'] = d3_state_2['贷款剩余天数'].apply(lambda x: 1 if x>0 else 0 )

#计算复杂特征

d3_state_2_agg = {
    '历史申请编号':['count'],
    '申请额度':['mean', 'max'],
    '信用额度':['mean', 'max'],
    '商品价格':['mean'],
    '信用额度差':['mean'],
    '信用额度比':['mean', 'max'],
    '贷款剩余天数':['max'], #最长未还天数
    '贷款未还完':['sum'],
    '利率1':['mean'],
    '利率2':['mean']
}
# 离散型
cat_agg = ['sum', 'mean']
for col1 in d3_state_2.columns:
    for col2 in cat_to_dummy:
        if (col2 in col1):
            d3_state_2_agg[col1] = cat_agg

d3_state_2 = d3_state_2.groupby('申请编号').agg(d3_state_2_agg)
# 还原列名
d3_state_2.columns = pd.Series(d3_state_2.columns.tolist()).apply(pd.Series).sum(axis=1)
name_list = {}
for col in d3_state_2.columns:
    name_list[col] = 'd3_st2'+col
d3_state_2.rename(columns=name_list,inplace=True)

if (fill_data):
    d3_state_2 = fill_median(d3_state_2,['贷款剩余天数max'])
    
d3_state_2.shape
d3_state_2.head()

(113550, 31)

initial num of cols: 31
客户类型has4
收益级别has5


,d3_st2历史申请编号count_x,d3_st2贷款年金mean,d3_st2贷款年金max,d3_st2贷款年金min,d3_st2申请额度mean_x,d3_st2申请额度max_x,d3_st2申请额度min,d3_st2信用额度mean_x,d3_st2信用额度max_x,d3_st2信用额度min,...,d3_st2收益级别_0sum,d3_st2收益级别_0mean,d3_st2收益级别_1sum,d3_st2收益级别_1mean,d3_st2收益级别_2sum,d3_st2收益级别_2mean,d3_st2收益级别_3sum,d3_st2收益级别_3mean,d3_st2收益级别_4sum,d3_st2收益级别_4mean
申请编号,,,,,,,,,,,,,,,,,,,,,
0,1,11.000000,11.0,11.0,10.00,10.0,10.0,9.000000,9.0,9.0,...,0.0,0.000,1.0,1.000000,0.0,0.0,0.0,0.000000,0.0,0.000000
1,2,8.000000,10.0,6.0,5.00,6.0,4.0,6.500000,8.0,5.0,...,0.0,0.000,2.0,1.000000,0.0,0.0,0.0,0.000000,0.0,0.000000
2,10,42.142857,74.0,14.0,49.00,79.0,28.0,50.750000,79.0,28.0,...,3.0,0.375,5.0,0.625000,0.0,0.0,0.0,0.000000,0.0,0.000000
3,1,49.000000,49.0,49.0,51.00,51.0,51.0,49.000000,49.0,49.0,...,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.000000,1.0,1.000000
4,9,17.666667,26.0,4.0,25.00,40.0,3.0,25.555556,40.0,3.0,...,0.0,0.000,4.0,0.444444,0.0,0.0,1.0,0.111111,4.0,0.444444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182975,3,12.333333,14.0,11.0,9.00,12.0,6.0,15.333333,28.0,6.0,...,0.0,0.000,1.0,0.500000,0.0,0.0,0.0,0.000000,1.0,0.500000
182976,2,27.000000,27.0,27.0,24.00,24.0,24.0,25.000000,25.0,25.0,...,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.000000,1.0,1.000000
182977,1,5.000000,5.0,5.0,5.00,5.0,5.0,5.000000,5.0,5.0,...,0.0,0.000,0.0,0.000000,1.0,1.0,0.0,0.000000,0.0,0.000000


(49287, 31)

,历史申请编号count,申请额度mean,申请额度max,信用额度mean,信用额度max,商品价格mean,信用额度差mean,信用额度比mean,信用额度比max,贷款剩余天数max,...,收益级别_0sum,收益级别_0mean,收益级别_1sum,收益级别_1mean,收益级别_2sum,收益级别_2mean,收益级别_3sum,收益级别_3mean,收益级别_4sum,收益级别_4mean
申请编号,,,,,,,,,,,,,,,,,,,,,
2,5,50699.000000,79211,52894.200000,79211.0,50699.000000,2195.200000,1.038,1.14,687.0,...,3,0.600000,2,0.400000,0,0.0,0,0.0,0,0.0
4,5,35651.000000,39611,35651.000000,39611.0,35651.000000,0.000000,1.000,1.00,-735.0,...,0,0.000000,1,0.200000,0,0.0,0,0.0,4,0.8
5,1,47531.000000,47531,52674.000000,52674.0,47531.000000,5143.000000,1.110,1.11,381.0,...,0,0.000000,1,1.000000,0,0.0,0,0.0,0,0.0
6,5,30375.000000,39611,31406.800000,39611.0,30375.000000,1031.800000,1.044,1.11,538.0,...,1,0.200000,0,0.000000,1,0.2,0,0.0,3,0.6
13,3,15982.333333,39611,22464.666667,59468.0,15982.333333,6482.333333,1.130,1.50,686.0,...,1,0.333333,2,0.666667,0,0.0,0,0.0,0,0.0


In [89]:
d3 = pd.merge(d3, d3_state_2, how='left', on='申请编号')

if (fill_data):
    d3.fillna(0,inplace=True)

d3.columns.to_list()


['历史申请编号count_x',
 '贷款年金mean',
 '贷款年金max',
 '贷款年金min',
 '申请额度mean_x',
 '申请额度max_x',
 '申请额度min',
 '信用额度mean_x',
 '信用额度max_x',
 '信用额度min',
 '商品价格mean_x',
 '商品价格max',
 '商品价格min',
 '信用额度差mean_x',
 '信用额度比mean_x',
 '信用额度比max_x',
 '历史贷款授信距本次申请时间mean',
 '历史贷款授信距本次申请时间max',
 '历史贷款授信距本次申请时间min',
 '历史贷款授信月数mean',
 '历史贷款授信月数max',
 '历史贷款授信月数min',
 '拒绝原因_0sum',
 '拒绝原因_0mean',
 '拒绝原因_1sum',
 '拒绝原因_1mean',
 '拒绝原因_2sum',
 '拒绝原因_2mean',
 '拒绝原因_3sum',
 '拒绝原因_3mean',
 '拒绝原因_4sum',
 '拒绝原因_4mean',
 '拒绝原因_5sum',
 '拒绝原因_5mean',
 '拒绝原因_6sum',
 '拒绝原因_6mean',
 '拒绝原因_7sum',
 '拒绝原因_7mean',
 '拒绝原因_8sum',
 '拒绝原因_8mean',
 '客户类型_0sum_x',
 '客户类型_0mean_x',
 '客户类型_1sum_x',
 '客户类型_1mean_x',
 '客户类型_2sum_x',
 '客户类型_2mean_x',
 '客户类型_3sum_x',
 '客户类型_3mean_x',
 '商品类别_0sum',
 '商品类别_0mean',
 '商品类别_1sum',
 '商品类别_1mean',
 '商品类别_2sum',
 '商品类别_2mean',
 '商品类别_3sum',
 '商品类别_3mean',
 '商品类别_4sum',
 '商品类别_4mean',
 '商品类别_5sum',
 '商品类别_5mean',
 '商品类别_6sum',
 '商品类别_6mean',
 '商品类别_7sum',
 '商品类别_7mean',
 '商品类别_8sum',
 '商品类别_8mean',
 '商品类别_9su

In [73]:
for col in d3.columns:
    d3.loc[d3[col]==np.inf, col] = np.NaN
    if (fill_data):
        d3.fillna(0,inplace=True)

print_na(d3)

KeyError: "None of [Index([('历', '史', '申', '请', '编', '号', 'c', 'o', 'u', 'n', 't', '_', 'x'), ('历', '史', '申', '请', '编', '号', 'c', 'o', 'u', 'n', 't', '_', 'x')], dtype='object', name='申请编号')] are in the [index]"

In [15]:
### d3 自定义特征

# print_val(d3,'历史贷款授信距本次申请时间')

# d3.shape
# '''d3 nan值'''
# print_na(d3)
# d3['信用额度比'].describe()
# print_val(d3,'信用额度比')

# d3['拟实际应还贷款金额'].describe()
# print_val(d3,'拟实际应还贷款金额')

(744799, 40)

'd3 nan值'

贷款类型 191
贷款年金 163686
申请额度 172441
信用额度 146654
首付金额 566356
商品价格 172434
首付率 396776
利率1 742192
利率2 742192
陪同申请人 368008
贷款期数 163009
产品组合 191
首付时间 715918
拟实际应还贷款金额 163686
信用额度比 172442
首付月数 715918


In [17]:
### d3 处理col含有过多值

# if ('销售区域' in d3.columns):
#     d3.drop(columns='销售区域',inplace=True)
#     d3a.drop(columns='销售区域',inplace=True)
#     d3d.drop(columns='销售区域',inplace=True)

In [18]:
### d3 group by ###

# 离散特征
# d3_cat_cols = [
#     '贷款类型',
#     '贷款用途',
#     '合同状态',
#     '付款方式',
# #     '拒绝原因',
#     '陪同申请人',
#     '客户类型',
#     '商品类别',
#     '组合类型',
#     '产品类型',
#     '获客渠道',
# #     '销售区域',
#     '行业',
#     '收益级别',
#     '产品组合',
#     # 时点型
#     '申请周内日',
#     '申请时点']

# d3_agg_dict = {
#     '历史申请编号':['count'],
#     # 数值型 - 金额
#     '贷款年金':['mean', 'max', 'min', 'sum'],
#     '申请额度':['mean', 'max', 'min', 'sum'],
#     '信用额度':['mean', 'max', 'min', 'sum'],
#     '首付金额':['mean', 'max', 'min'],
#     '商品价格':['mean', 'max', 'min'],
#     '拟实际应还贷款金额':['mean','max', 'min', 'sum'],
#     '信用额度比':['mean', 'max', 'min'],
#     # 数值型 - 时长 
#     '历史贷款授信距本次申请时间':['mean', 'max', 'min'],
#     '历史贷款授信月数':['mean', 'max', 'min'],
#     '首付时间':['mean', 'max', 'min'],
#     '首付月数':['mean', 'max', 'min'],
#     # 概率型
#     '首付率':['mean', 'max', 'min'],
#     '利率1':['mean', 'max', 'min'],
#     '利率2':['mean', 'max', 'min']
# }
    
# # 离散型
# cat_agg = ['sum', 'mean']
# for col1 in d3.columns:
#     for col2 in d3_cat_cols:
#         if (col2 in col1):
#             d3_agg_dict[col1] = cat_agg

# # 非拒合同状态group
# d3a = d3a.groupby('申请编号').agg(d3_agg_dict)
# # 还原列名
# d3a.columns = pd.Series(d3a.columns.tolist()).apply(pd.Series).sum(axis=1)

# # 拒绝合同状态group
# d3d = d3d.groupby('申请编号').agg(d3_agg_dict)
# # 还原列名
# d3d.columns = pd.Series(d3d.columns.tolist()).apply(pd.Series).sum(axis=1)

# # d3all
# d3all_agg_dict = {
#     '历史申请编号':['count'],
#     '贷款年金':['sum'],
#     '申请额度':['sum'],
#     '信用额度':['sum']
# }

# # groupby全部
# d3all = d3.groupby('申请编号').agg(d3all_agg_dict)
# # 还原列名
# d3all.columns = pd.Series(d3all.columns.tolist()).apply(pd.Series).sum(axis=1)

  
# d3a.shape
# # d3a.columns.to_list()
# d3d.shape
# # d3d.columns.to_list()
# d3all.shape
# # d3all.columns.to_list()

(152722, 331)

(53621, 331)

(152954, 4)

In [19]:


# if (fill_data):
#     for df in [d3a,d3d,d3]:
#         for col in df.columns.to_list():
#             df.fillna(df[col].median(),inplace=True)
    

# '''d3a'''
# print_na(d3a)
# '''d3d'''
# print_na(d3d)
# '''d3'''
# print_na(d3)


'd3a'

贷款年金mean 265
贷款年金max 265
贷款年金min 265
申请额度mean 671
申请额度max 671
申请额度min 671
信用额度mean 167
信用额度max 167
信用额度min 167
首付金额mean 53218
首付金额max 53218
首付金额min 53218
商品价格mean 671
商品价格max 671
商品价格min 671
拟实际应还贷款金额mean 265
拟实际应还贷款金额max 265
拟实际应还贷款金额min 265
信用额度比mean 671
信用额度比max 671
信用额度比min 671
首付时间mean 123915
首付时间max 123915
首付时间min 123915
首付月数mean 123915
首付月数max 123915
首付月数min 123915
首付率mean 9964
首付率max 9964
首付率min 9964
利率1mean 150213
利率1max 150213
利率1min 150213
利率2mean 150213
利率2max 150213
利率2min 150213


'd3d'

贷款年金mean 4692
贷款年金max 4692
贷款年金min 4692
申请额度mean 4334
申请额度max 4334
申请额度min 4334
信用额度mean 2645
信用额度max 2645
信用额度min 2645
首付金额mean 42376
首付金额max 42376
首付金额min 42376
商品价格mean 4332
商品价格max 4332
商品价格min 4332
拟实际应还贷款金额mean 4692
拟实际应还贷款金额max 4692
拟实际应还贷款金额min 4692
信用额度比mean 4334
信用额度比max 4334
信用额度比min 4334
首付时间mean 53621
首付时间max 53621
首付时间min 53621
首付月数mean 53621
首付月数max 53621
首付月数min 53621
首付率mean 29655
首付率max 29655
首付率min 29655
利率1mean 53621
利率1max 53621
利率1min 53621
利率2mean 53621
利率2max 53621
利率2min 53621


'd3'

贷款年金 163686
申请额度 172441
信用额度 146654
首付金额 566356
商品价格 172434
首付率 396776
利率1 742192
利率2 742192
贷款期数 163009
首付时间 715918
拟实际应还贷款金额 163686
信用额度比 172442
首付月数 715918


In [20]:
# len(d3a.index.to_list())
# d3a_id = set(d3a.index.to_list())
# len(d3a_id)

# len(d3d.index.to_list())
# d3d_id = set(d3d.index.to_list())
# len(d3d_id)

# inter = d3a_id.intersection(d3d_id)
# len(inter)

### intersection
# ''' d01&d02 '''
# id_train = set(d01['申请编号'])
# len(id_train)
# id_test = set(d02['申请编号'])
# len(id_test)
# inter = id_train.intersection(id_test)
# len(inter)

152722

152722

53621

53621

53389

In [21]:
# ### d3all merge d3a
# d3all_d3a = pd.merge(d3all, d3a, how='left', on='申请编号')

# # d3a_all自定义特征
# d3all_d3a.eval('拟申请核准率_a = 历史申请编号count_y / 历史申请编号count_x', inplace=True)
# d3all_d3a.eval('贷款年金sum比_a = 贷款年金sum_y / 贷款年金sum_x', inplace=True)
# d3all_d3a.eval('申请额度sum比_a = 申请额度sum_y / 申请额度sum_x', inplace=True)
# d3all_d3a.eval('信用额度sum比_a = 信用额度sum_y / 信用额度sum_x', inplace=True)
# d3all_d3a.eval('信用比申请额度_a = 信用额度sum_y / 申请额度sum_y', inplace=True)

# d3all_d3a.shape
# d3all_d3a.columns.to_list()

(152954, 340)

['历史申请编号count_x',
 '贷款年金sum_x',
 '申请额度sum_x',
 '信用额度sum_x',
 '历史申请编号count_y',
 '贷款年金mean',
 '贷款年金max',
 '贷款年金min',
 '贷款年金sum_y',
 '申请额度mean',
 '申请额度max',
 '申请额度min',
 '申请额度sum_y',
 '信用额度mean',
 '信用额度max',
 '信用额度min',
 '信用额度sum_y',
 '首付金额mean',
 '首付金额max',
 '首付金额min',
 '商品价格mean',
 '商品价格max',
 '商品价格min',
 '拟实际应还贷款金额mean',
 '拟实际应还贷款金额max',
 '拟实际应还贷款金额min',
 '拟实际应还贷款金额sum',
 '信用额度比mean',
 '信用额度比max',
 '信用额度比min',
 '历史贷款授信距本次申请时间mean',
 '历史贷款授信距本次申请时间max',
 '历史贷款授信距本次申请时间min',
 '历史贷款授信月数mean',
 '历史贷款授信月数max',
 '历史贷款授信月数min',
 '首付时间mean',
 '首付时间max',
 '首付时间min',
 '首付月数mean',
 '首付月数max',
 '首付月数min',
 '首付率mean',
 '首付率max',
 '首付率min',
 '利率1mean',
 '利率1max',
 '利率1min',
 '利率2mean',
 '利率2max',
 '利率2min',
 '贷款类型_0sum',
 '贷款类型_0mean',
 '贷款类型_1sum',
 '贷款类型_1mean',
 '贷款类型_Asum',
 '贷款类型_Amean',
 '贷款用途_0sum',
 '贷款用途_0mean',
 '贷款用途_1sum',
 '贷款用途_1mean',
 '贷款用途_2sum',
 '贷款用途_2mean',
 '贷款用途_3sum',
 '贷款用途_3mean',
 '贷款用途_4sum',
 '贷款用途_4mean',
 '贷款用途_5sum',
 '贷款用途_5mean',
 '贷款用途_6sum',
 '贷款用途_6mean',
 '贷款用途_

In [22]:
# drop_list = ['贷款年金sum_y', '申请额度sum_y', '信用额度sum_y', '历史申请编号count_y']
# d3all_d3a.drop(columns=drop_list,inplace=True)

In [23]:
# ### d3all_d3a merge d3d
# d3all_a_d = pd.merge(d3all_d3a, d3d, how='left', on='申请编号')

# # d3all_a_d
# d3all_a_d.eval('拟申请拒绝率_d = 历史申请编号count / 历史申请编号count_x', inplace=True)
# d3all_a_d.eval('贷款年金sum比_d = 贷款年金sum / 贷款年金sum_x', inplace=True)
# d3all_a_d.eval('申请额度sum比_d = 申请额度sum / 申请额度sum_x', inplace=True)
# d3all_a_d.eval('信用额度sum比_d = 信用额度sum / 信用额度sum_x', inplace=True)
# d3all_a_d.eval('信用比申请额度_d = 信用额度sum / 申请额度sum', inplace=True)

# d3all_a_d.shape
# d3all_a_d.columns.to_list()

(152954, 672)

['历史申请编号count_x',
 '贷款年金sum_x',
 '申请额度sum_x',
 '信用额度sum_x',
 '贷款年金mean_x',
 '贷款年金max_x',
 '贷款年金min_x',
 '申请额度mean_x',
 '申请额度max_x',
 '申请额度min_x',
 '信用额度mean_x',
 '信用额度max_x',
 '信用额度min_x',
 '首付金额mean_x',
 '首付金额max_x',
 '首付金额min_x',
 '商品价格mean_x',
 '商品价格max_x',
 '商品价格min_x',
 '拟实际应还贷款金额mean_x',
 '拟实际应还贷款金额max_x',
 '拟实际应还贷款金额min_x',
 '拟实际应还贷款金额sum_x',
 '信用额度比mean_x',
 '信用额度比max_x',
 '信用额度比min_x',
 '历史贷款授信距本次申请时间mean_x',
 '历史贷款授信距本次申请时间max_x',
 '历史贷款授信距本次申请时间min_x',
 '历史贷款授信月数mean_x',
 '历史贷款授信月数max_x',
 '历史贷款授信月数min_x',
 '首付时间mean_x',
 '首付时间max_x',
 '首付时间min_x',
 '首付月数mean_x',
 '首付月数max_x',
 '首付月数min_x',
 '首付率mean_x',
 '首付率max_x',
 '首付率min_x',
 '利率1mean_x',
 '利率1max_x',
 '利率1min_x',
 '利率2mean_x',
 '利率2max_x',
 '利率2min_x',
 '贷款类型_0sum_x',
 '贷款类型_0mean_x',
 '贷款类型_1sum_x',
 '贷款类型_1mean_x',
 '贷款类型_Asum_x',
 '贷款类型_Amean_x',
 '贷款用途_0sum_x',
 '贷款用途_0mean_x',
 '贷款用途_1sum_x',
 '贷款用途_1mean_x',
 '贷款用途_2sum_x',
 '贷款用途_2mean_x',
 '贷款用途_3sum_x',
 '贷款用途_3mean_x',
 '贷款用途_4sum_x',
 '贷款用途_4mean_x',
 '贷款用途_

In [24]:

# drop_list = ['历史申请编号count_x', '贷款年金sum_x', '申请额度sum_x', '信用额度sum_x',
#              '历史申请编号count', '贷款年金sum', '申请额度sum', '信用额度sum']
# d3all_a_d.drop(columns=drop_list,inplace=True)

In [25]:
# ### merge d12 and d3a_all
# d12.shape
# d3all_a_d.shape
# d12_d3all_a_d = pd.merge(d12, d3all_a_d, how='left', on='申请编号')
# d12_d3all_a_d.shape
# print_na(d12_d3all_a_d)

# #TODO: 空值插补0
# d12_d3all_a_d = d12_d3all_a_d.fillna(0)

(161469, 127)

(152954, 664)

(161469, 791)

贷款年金 7
商品价格 137
工作日期距申请日期天数 26833
外部评分 90777
贷款申请前1小时内征信查询次数 22203
贷款申请前1天内征信查询次数 22203
贷款申请前1周内征信查询次数 22203
贷款申请前1个月内征信查询次数 22203
贷款申请前1个季度内征信查询次数 22203
贷款申请前1年内征信查询次数 22203
有车时间 106452
家庭成员数 1
居住信息1 107790
居住信息2 113104
居住信息3 81552
居住信息4 78505
最近一次换手机号码距申请日天数 25482
最近一次换手机号月数 25482
有车月数 106452
贷款年金mean_x 9012
贷款年金max_x 9012
贷款年金min_x 9012
申请额度mean_x 9418
申请额度max_x 9418
申请额度min_x 9418
信用额度mean_x 8914
信用额度max_x 8914
信用额度min_x 8914
首付金额mean_x 61965
首付金额max_x 61965
首付金额min_x 61965
商品价格mean_x 9418
商品价格max_x 9418
商品价格min_x 9418
拟实际应还贷款金额mean_x 9012
拟实际应还贷款金额max_x 9012
拟实际应还贷款金额min_x 9012
拟实际应还贷款金额sum_x 8747
信用额度比mean_x 9418
信用额度比max_x 9418
信用额度比min_x 9418
历史贷款授信距本次申请时间mean_x 8747
历史贷款授信距本次申请时间max_x 8747
历史贷款授信距本次申请时间min_x 8747
历史贷款授信月数mean_x 8747
历史贷款授信月数max_x 8747
历史贷款授信月数min_x 8747
首付时间mean_x 132662
首付时间max_x 132662
首付时间min_x 132662
首付月数mean_x 132662
首付月数max_x 132662
首付月数min_x 132662
首付率mean_x 18711
首付率max_x 18711
首付率min_x 18711
利率1mean_x 158960
利率1max_x 158960
利率1min_x 158960
利率2mean_x 1

In [26]:
### test

# inter_d12_d3all_a_d = d12_d3all_a_d.loc[inter]


# inter_d12_d3all_a_d.to_csv(f'./tmp/{prefix}_d1234.csv')

In [60]:
# # 检查d4 nan值
'''d4 nan值'''
print_na(d4)

# # d4.head()
# d0 = pd.read_csv('./data/train_label.csv')
# # d0.shape
# # d0.head()

# d40 = pd.merge(d4, d0, how='left', on='申请编号')

# d40['分期付款日期'].value_counts()
# pd.DataFrame(d40.loc[d40['标签']==0,'分期付款日期'].value_counts().sort_index())

# pd.DataFrame(d40.loc[d40['分期付款日期']==np.NaN,'标签'].value_counts().sort_index())

# d4.groupby(['申请编号','历史申请编号','有支付行为的分期付款']).agg({'本期贷款金额':'sum'}).reset_index()


'd4 nan值'

分期付款实际支付时间 1542
本期还款金额 1542


In [61]:
### d4 缺失值插补

# for col in d4.columns:
#     d4[col].value_counts()

# 距申请日期>0, 置平均值
# d4_cols = {}
# d4_cols['_dt'] = [
#     '分期付款实际支付时间']
    
# nan_to_mean = [
#     '分期付款实际支付时间',
#     '本期还款金额'
# ]

#TODO 空值插补0
d4 = d4.fillna(0)

In [62]:
### d4 自定义特征

d4.eval('time_diff = 分期付款实际支付时间 - 分期付款应支付时间', inplace=True)

d4['is_odue'] = d4['time_diff'].apply(lambda x: 1 if x>0 else 0)
d4['is_odue30'] = d4['time_diff'].apply(lambda x: 1 if (x>=30) else 0)
d4['is_odue60'] = d4['time_diff'].apply(lambda x: 1 if (x>=60) else 0)
d4['is_odue90'] = d4['time_diff'].apply(lambda x: 1 if (x>=90) else 0)


# d4['分期付款应支付时间'].value_counts()
# d4['分期付款实际支付时间'].value_counts()

In [63]:
### d4 group by phase 1 ###

d41_agg_dict = {
    '本期贷款金额':['sum'],
    '本期还款金额':['mean'],
    'time_diff':['mean'],
    'is_odue':['sum'],
    'is_odue30':['sum'],
    'is_odue60':['sum'],
    'is_odue90':['sum']
}
d41 = d4.groupby(['申请编号','历史申请编号','有支付行为的分期付款']).agg(d41_agg_dict)
d41.columns = pd.Series(d41.columns.tolist()).apply(pd.Series).sum(axis=1)
d41.shape
d41.head()


(5713189, 7)

本期贷款金额sum  本期还款金额mean  time_diffmean  is_oduesum  \
申请编号 历史申请编号 有支付行为的分期付款                                                     
0    779169 2              1100.0      1100.0          -14.0           0   
            3              1100.0      1100.0          -13.0           0   
            4              1100.0      1100.0            1.0           1   
            5              1100.0      1100.0          -23.0           0   
            7              1100.0      1100.0           -1.0           0   

                        is_odue30sum  is_odue60sum  is_odue90sum  
申请编号 历史申请编号 有支付行为的分期付款                                            
0    779169 2                      0             0             0  
            3                      0             0             0  
            4                      0             0             0  
            5                      0             0             0  
            7                      0             0             0

In [64]:
### d4 group by phase 2 ###

d42_agg_dict = {
    '本期还款金额mean':['sum'],
    '本期贷款金额sum':['sum'],
    'time_diffmean':['mean','max'],
    'is_oduesum':['sum'], # sum-累计逾期
    'is_odue30sum':['sum'], # sum-累计逾期
    'is_odue60sum':['sum'], # sum-累计逾期
    'is_odue90sum':['sum'] # sum-累计逾期
}
d42 = d41.groupby(['申请编号','历史申请编号']).agg(d42_agg_dict)
d42.columns = pd.Series(d42.columns.tolist()).apply(pd.Series).sum(axis=1)
d42.shape
d42.head()


(445568, 8)

本期还款金额meansum  本期贷款金额sumsum  time_diffmeanmean  time_diffmeanmax  \
申请编号 历史申请编号                                                                     
0    779169        11538.0       16472.0         -11.000000               1.0   
1    313225         7428.5        8376.0          -6.250000              -1.0   
     753009        11712.0       11712.0          -3.083333               0.0   
2    702654        57934.0       65493.0          -2.458333               0.0   
     710924        38610.0       38610.0          -3.466667               1.0   

             is_oduesumsum  is_odue30sumsum  is_odue60sumsum  is_odue90sumsum  
申请编号 历史申请编号                                                                    
0    779169              4                0                0                0  
1    313225              1                0                0                0  
     753009              0                0                0                0  
2    702654              1                0                0                0  
     710924              1                0                0                0

In [65]:
### d4 自定义特征

d42.eval('还款金额比贷款金额 = 本期还款金额meansum / 本期贷款金额sumsum', inplace=True)
d42.columns.to_list()

['本期还款金额meansum',
 '本期贷款金额sumsum',
 'time_diffmeanmean',
 'time_diffmeanmax',
 'is_oduesumsum',
 'is_odue30sumsum',
 'is_odue60sumsum',
 'is_odue90sumsum',
 '还款金额比贷款金额']

In [66]:
### d4 group by phase 3 ###

d43 = d42.groupby('申请编号').agg({
    '还款金额比贷款金额':['mean'],
    '本期还款金额meansum':['sum'],
    '本期贷款金额sumsum':['sum'],
    'time_diffmeanmean':['mean'], 
    'time_diffmeanmax':['max'],
    'is_oduesumsum':['sum'],
    'is_odue30sumsum':['sum'],
    'is_odue60sumsum':['sum'],
    'is_odue90sumsum':['sum']
})
d43.columns = pd.Series(d43.columns.tolist()).apply(pd.Series).sum(axis=1)
'''d43'''
d43.shape
d43.head()


'd43'

(153256, 9)

,还款金额比贷款金额mean,本期还款金额meansumsum,本期贷款金额sumsumsum,time_diffmeanmeanmean,time_diffmeanmaxmax,is_oduesumsumsum,is_odue30sumsumsum,is_odue60sumsumsum,is_odue90sumsumsum
申请编号,,,,,,,,,
0,0.700461,11538.00,16472.0,-11.000000,1.00,4,0,0,0
1,0.943440,19140.50,20088.0,-4.666667,0.00,1,0,0,0
2,0.971146,221145.00,228704.0,-1.981250,1.00,2,0,0,0
3,1.000000,58938.00,58938.0,-15.000000,-12.00,0,0,0,0
4,0.818884,67489.25,94053.0,-4.096131,42.25,18,3,1,0


In [67]:
### d4 group by phase 4 ###

d4_agg_dict = {}
d4_cols = {}
# 金额型
d4_cols['curr'] = [
    '本期贷款金额',
    '本期还款金额']


# 时长型
d4_cols['time_span'] = [
    '分期付款应支付时间',
    '分期付款实际支付时间']

# 计数型
d4_cols['cnt'] = [
    '有支付行为的分期付款']

# 数值型 - 金额
curr_agg = ['mean', 'max', 'min']
for col in d4_cols['curr']:
    d4_agg_dict[col] = curr_agg

# 数值型 - 时长 
time_span_agg = ['mean', 'max', 'min']
for col in d4_cols['time_span']:
    d4_agg_dict[col] = time_span_agg

d4 = d4.groupby('申请编号').agg(d4_agg_dict)
d4.columns = pd.Series(d4.columns.tolist()).apply(pd.Series).sum(axis=1)
'''d4'''
d4.shape

d4 = pd.merge(d4, d43, how='inner', on='申请编号')
'''d4'''
d4.shape
d4.head()

'd4'

(153256, 12)

'd4'

(153256, 21)

,本期贷款金额mean,本期贷款金额max,本期贷款金额min,本期还款金额mean,本期还款金额max,本期还款金额min,分期付款应支付时间mean,分期付款应支付时间max,分期付款应支付时间min,分期付款实际支付时间mean,...,分期付款实际支付时间min,还款金额比贷款金额mean,本期还款金额meansumsum,本期贷款金额sumsumsum,time_diffmeanmeanmean,time_diffmeanmaxmax,is_oduesumsumsum,is_odue30sumsumsum,is_odue60sumsumsum,is_odue90sumsumsum
申请编号,,,,,,,,,,,,,,,,,,,,,
0,1098.133333,1100.0,1072.0,880.266667,1100.0,12.0,-1600.000000,-1480,-1744,-1610.333333,...,-1758.0,0.700461,11538.00,16472.0,-11.000000,1.00,4,0,0,0
1,803.520000,976.0,635.0,778.520000,976.0,242.0,-267.920000,-19,-518,-272.480000,...,-527.0,0.943440,19140.50,20088.0,-4.666667,0.00,1,0,0,0
2,5578.146341,39611.0,2574.0,5455.414634,39611.0,607.0,-331.829268,-4,-641,-334.439024,...,-648.0,0.971146,221145.00,228704.0,-1.981250,1.00,2,0,0,0
3,4911.500000,4912.0,4906.0,4911.500000,4912.0,4906.0,-252.000000,-120,-384,-267.000000,...,-396.0,1.000000,58938.00,58938.0,-15.000000,-12.00,0,0,0,0
4,1383.132353,1646.0,348.0,1111.882353,1645.0,14.0,-1560.852941,-1179,-2296,-1562.161765,...,-2305.0,0.818884,67489.25,94053.0,-4.096131,42.25,18,3,1,0


In [71]:
### Merge d12 and d3 ###
'''d12'''
d12.shape
'''d3'''
d3.shape

d12_d3 = pd.merge(d12, d3, how='left', on='申请编号')

'''d12_d3'''
d12_d3.shape
print_na(d12_d3)

#TODO 空值插补0
# d12_d4 = d12_d4.fillna(0)

'd12'

(161469, 187)

'd3'

(153256, 21)

'd12_d3'

(161469, 476)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [68]:
### Merge d12 and d4 ###
'''d12'''
d12.shape
'''d4'''
d4.shape

d12_d4 = pd.merge(d12['申请编号'], d4, how='left', on='申请编号')

'''d12_d4'''
d12_d4.shape
print_na(d12_d4)

#TODO 空值插补0
d12_d4 = d12_d4.fillna(0)

'd12'

(161469, 187)

'd4'

(153256, 21)

'd12_d4'

(161469, 22)

本期贷款金额mean 8213
本期贷款金额max 8213
本期贷款金额min 8213
本期还款金额mean 8213
本期还款金额max 8213
本期还款金额min 8213
分期付款应支付时间mean 8213
分期付款应支付时间max 8213
分期付款应支付时间min 8213
分期付款实际支付时间mean 8213
分期付款实际支付时间max 8213
分期付款实际支付时间min 8213
还款金额比贷款金额mean 8213
本期还款金额meansumsum 8213
本期贷款金额sumsumsum 8213
time_diffmeanmeanmean 8213
time_diffmeanmaxmax 8213
is_oduesumsumsum 8213
is_odue30sumsumsum 8213
is_odue60sumsumsum 8213
is_odue90sumsumsum 8213


In [69]:
### Merge d12_d3a_all and d12_d4 ###

d3.shape
d12_d4.shape
d1234 = pd.merge(d12_d3all_a_d, d12_d4, how='left', on='申请编号')
d1234.shape
print_na(d1234)

NameError: name 'd12_d3all_a_d' is not defined

In [37]:
## Deal with Inf

for f in d1234.columns:
    d1234.loc[d1234[f]==np.inf, f] = np.NaN
    if (fill_data):
        d1234.fillna(d1234[f].median(),inplace=True)
        
d1234.shape

(161469, 812)

In [38]:
### BoxCox

def get_boxcox(x):
    inp = x.apply('abs') + 1
    l, opt_lmbda = spstats.boxcox(inp)
    opt_lmbda
    lmbda = 0
    if opt_lmbda > 0:
        lmbda = opt_lmbda
    ret = spstats.boxcox(inp, lmbda=lmbda)
    return lmbda, ret

# Features
large_feats_drop = ['客户收入', '申请编号']
large_feats = d1234.max()[d1234.max()>100].index.drop(large_feats_drop)

lmbda = {}
for f in large_feats:
    lmbda[f], d1234[f] = get_boxcox(d1234[f])

In [39]:
prefix = 'train'
data_train = pd.merge(d01['申请编号'], d1234, on='申请编号', how='left')
data_train.shape
data_train.to_csv(f'./tmp/{prefix}_d1234.csv')

prefix = 'A'
data_test = pd.merge(d05['申请编号'], d1234, on='申请编号', how='left')
data_test.shape
data_test.to_csv(f'./tmp/{prefix}_d1234.csv')

(140000, 812)

(21469, 812)

In [40]:
data_train.head()

,申请编号,贷款类型,信用额度,贷款年金,商品价格,出生日期距申请日期天数,工作日期距申请日期天数,注册日期距申请日期天数,身份认证日期距申请日期天数,是否提供手机号,...,分期付款实际支付时间min,还款金额比贷款金额mean,本期还款金额meansumsum,本期贷款金额sumsumsum,time_diffmeanmeanmean,time_diffmeanmaxmax,is_oduesumsumsum,is_odue30sumsumsum,is_odue60sumsumsum,is_odue90sumsumsum
0,0,0.0,13.039399,265.369224,42.0,-28.0,-1.0,-5421.0,-110.0,1.0,...,-1758.0,0.547088,39.404465,43.913948,3.495124,0.693147,1.609438,0.0,0.0,0.0
1,1,0.0,12.958365,250.542447,38.0,-38.0,-3.0,-3387.0,-28.0,1.0,...,-527.0,0.689946,45.440482,46.434931,2.192964,0.000000,0.693147,0.0,0.0,0.0
2,2,0.0,13.059065,275.722615,42.0,-37.0,-6.0,-3870.0,-6.0,1.0,...,-648.0,0.705213,88.832316,90.599786,1.263651,0.693147,1.098612,0.0,0.0,0.0
3,3,0.0,13.048048,259.874019,41.0,-45.0,0.0,-970.0,-97.0,0.0,...,-396.0,0.720912,62.038823,62.614097,4.065916,2.564949,0.000000,0.0,0.0,0.0
4,5,0.0,12.981256,255.211944,39.0,-49.0,-30.0,-9350.0,-120.0,1.0,...,-2306.0,0.679868,59.769084,60.764259,2.125462,0.000000,0.693147,0.0,0.0,0.0


In [41]:
'''All Done'''
data_train.columns.to_list()

'All Done'

['申请编号',
 '贷款类型',
 '信用额度',
 '贷款年金',
 '商品价格',
 '出生日期距申请日期天数',
 '工作日期距申请日期天数',
 '注册日期距申请日期天数',
 '身份认证日期距申请日期天数',
 '是否提供手机号',
 '是否提供电话',
 '手机号是否有效',
 '是否提供email',
 '申请周内日',
 '申请时点',
 '外部评分',
 '申请人是否额外提供了文件2',
 '申请人是否额外提供了文件3',
 '申请人是否额外提供了文件4',
 '申请人是否额外提供了文件5',
 '申请人是否额外提供了文件6',
 '申请人是否额外提供了文件7',
 '申请人是否额外提供了文件8',
 '申请人是否额外提供了文件9',
 '贷款申请前1小时内征信查询次数',
 '贷款申请前1天内征信查询次数',
 '贷款申请前1周内征信查询次数',
 '贷款申请前1个月内征信查询次数',
 '贷款申请前1个季度内征信查询次数',
 '贷款申请前1年内征信查询次数',
 '性别',
 '是否有车',
 '是否有房',
 '孩子个数',
 '客户收入',
 '婚姻状态',
 '居住地人口密集度',
 '有车时间',
 '家庭成员数',
 '居住信息1',
 '居住信息2',
 '居住信息3',
 '居住信息4',
 '客户居住地评分1',
 '客户居住地评分2',
 '地址是否一致标志1',
 '地址是否一致标志2',
 '地址是否一致标志3',
 '地址是否一致标志4',
 '地址是否一致标志5',
 '地址是否一致标志6',
 '社交圈违约信息2_2',
 '社交圈违约信息2_1',
 '社交圈违约信息1_2',
 '社交圈违约信息1_1',
 '最近一次换手机号码距申请日天数',
 '有外部评分',
 '最近一次换手机号月数',
 '一月内换过手机号',
 '25岁以下',
 '25岁到30岁',
 '30岁到40岁',
 '40岁到50岁',
 '50岁以上',
 '工龄小于2年',
 '注册日期距申请日期月数',
 '有车月数',
 '商品价格小于42',
 '客户居住地评分sum',
 '客户居住地评分mean',
 '地址是否一致标志sum',
 '地址是否一致标志mean',
 '有居住信息1',
 '有居住信息2',
 '有居住信息